# Span Batch Analysis - Data Preparation

## Initialization

Before start, Fill in [secret.yml](span-batch-analysis/secret.yaml). Refer to [secret_example.yml](span-batch-analysis/secret_example.yaml). This is for specifying RPC endpoints.

Also check config located at [config](span-batch-analysis/config/).

In [2]:
import os

Build binaries.

In [3]:
%%bash
cd cmd/batch_decoder && go build . && cd ../../
cd cmd/span_batch_tester && go build . && cd ../../

In [4]:
%%capture
%env BATCH_DECODER=cmd/batch_decoder/batch_decoder
%env SPAN_BATCH_TESTER=cmd/span_batch_tester/span_batch_tester

Clone superchain config.

In [5]:
%%bash
git clone https://github.com/ethereum-optimism/superchain-registry
cd superchain-registry && git checkout 5d3ca1966435827ac8c4216bddcb541ec64c1d6c && cd --

fatal: destination path 'superchain-registry' already exists and is not an empty directory.


HEAD is now at 5d3ca19 superchain/extra: include minified genesis state data (#2)


Select your chain.

In [6]:
%%capture
%env L1_CHAIN_NAME=mainnet
%env L2_CHAIN_NAME=op

# available networks
#      L1 - L2 list
# mainnet - base, op, pgn, zora
# goerli  - base, op, zora
# sepolia - op, pgn

In [7]:
l1_chain_name = os.environ["L1_CHAIN_NAME"]
l2_chain_name = os.environ["L2_CHAIN_NAME"]

start_number = !yq eval ".start_l1_number" "span-batch-analysis/config/{l1_chain_name}/{l2_chain_name}.yaml"
start_number_value = int(start_number[0])
end_number = !yq eval ".end_l1_number" "span-batch-analysis/config/{l1_chain_name}/{l2_chain_name}.yaml"
end_number_value = int(end_number[0])
batch_tx_sender = !yq eval ".batch_tx_sender" "span-batch-analysis/config/{l1_chain_name}/{l2_chain_name}.yaml"
batch_tx_sender_value = batch_tx_sender[0]
l2_genesis_timestamp = !yq eval ".genesis.l2_time" "superchain-registry/superchain/configs/{l1_chain_name}/{l2_chain_name}.yaml"
l2_genesis_timestamp_value = int(l2_genesis_timestamp[0])
l2_chain_id = !yq eval ".chain_id" "superchain-registry/superchain/configs/{l1_chain_name}/{l2_chain_name}.yaml"
l2_chain_id_value = int(l2_chain_id[0])
inbox_addr = !yq eval ".batch_inbox_addr" "superchain-registry/superchain/configs/{l1_chain_name}/{l2_chain_name}.yaml"
inbox_addr_value = inbox_addr[0]

acc_start_number = !yq eval ".acc_l2_start_number" "span-batch-analysis/config/{l1_chain_name}/{l2_chain_name}.yaml"
acc_start_number_value = int(acc_start_number[0])
acc_end_number = !yq eval ".acc_l2_end_number" "span-batch-analysis/config/{l1_chain_name}/{l2_chain_name}.yaml"
acc_end_number_value = int(acc_end_number[0])

Read analyis config.

In [8]:
%env L1_START=$start_number_value
%env L1_END=$end_number_value
%env BATCH_TX_SENDER=$batch_tx_sender_value
%env L2_GENESIS_TIMESTAMP=$l2_genesis_timestamp_value
%env L2_CHAIN_ID=$l2_chain_id_value
%env INBOX=$inbox_addr_value

%env ACC_L2_START=$acc_start_number_value
%env ACC_L2_END=$acc_end_number_value


env: L1_START=17890000
env: L1_END=17910000
env: BATCH_TX_SENDER=0x6887246668a3b87F54DeB3b94Ba47a6f63F32985
env: L2_GENESIS_TIMESTAMP=1686068903
env: L2_CHAIN_ID=10
env: INBOX=0xff00000000000000000000000000000000000010
env: ACC_L2_START=107880000
env: ACC_L2_END=107881000


Get L1 RPC from secret.

In [9]:
key_name = f"L1_{l1_chain_name}_RPC".upper()
L1_RPC = !yq eval ".{key_name}" span-batch-analysis/secret.yaml
L1_RPC_ENDPOINT = L1_RPC[0]

key_name = f"L2_{l2_chain_name}_{l1_chain_name}_RPC".upper()
L2_RPC = !yq eval ".{key_name}" span-batch-analysis/secret.yaml
L2_RPC_ENDPOINT = L2_RPC[0]

In [10]:
%%capture
%env L1_RPC=$L1_RPC_ENDPOINT
%env L2_RPC=$L2_RPC_ENDPOINT

## Data Collection

All data will be stored at `./artifacts/{L1_CHAIN_NAME}/{L2_CHAIN_NAME}`

In [11]:
l1_chain_name = os.environ["L1_CHAIN_NAME"]
l2_chain_name = os.environ["L2_CHAIN_NAME"]

%env DB_PREFIX=./artifacts/$l1_chain_name/$l2_chain_name

env: DB_PREFIX=./artifacts/mainnet/op


batch decoder `fetch`: Read channel frames from L1 by reading batch transactions.

In [11]:
!$BATCH_DECODER fetch --l1=$L1_RPC --start=$L1_START --end=$L1_END --inbox=$INBOX --out=$DB_PREFIX/transactions_cache --sender=$BATCH_TX_SENDER --concurrent-requests=25 | tail -10

Fetched block:  17909997
Fetched block:  17909967
Fetched block:  17909968
Fetched block:  17909975
Fetched block:  17909981
Fetched block:  17909993
Fetched block:  17909982
Fetched batches in range [17890000,17910000). Found 2163 valid & 792 invalid batches
Fetch Config: Chain ID: 1. Inbox Address: 0xFF00000000000000000000000000000000000010. Valid Senders: map[0x6887246668a3b87F54DeB3b94Ba47a6f63F32985:{}].
Wrote transactions with batches to ./artifacts/mainnet/op/transactions_cache


batch decoder `reassemble`: Reassemble channel frames to channels. 

In [12]:
!$BATCH_DECODER reassemble --inbox=$INBOX --in=$DB_PREFIX/transactions_cache --out=$DB_PREFIX/channel_cache | tail -10

Channel dda230ad9d14cfeb0cd839fe4e05424f is not ready
Channel a396ab40b67c58c7a18c17b77c7bd73f is not ready


span batch tester `convert`: Convert channels with singular batches to span batches. Try multiple tx encoding types.

In [13]:
!TX_ENCODE_TYPE=0  && $SPAN_BATCH_TESTER convert --in=$DB_PREFIX/channel_cache --out=$DB_PREFIX/span_batch_cache_v$TX_ENCODE_TYPE --l2=$L2_RPC --genesis-timestamp=$L2_GENESIS_TIMESTAMP --chain-id=$L2_CHAIN_ID --tx-type=$TX_ENCODE_TYPE | tail -10

[401/410] Channel ID: fa729080158c4409583baa64ca47d871, L2StartNum: 108145198, L2EndNum: 108145492, L2BlockCnt, 295
[402/410] Channel ID: fb294776caf374b6b5ad4dbba5b68e28, L2StartNum: 108113345, L2EndNum: 108113644, L2BlockCnt, 300
[403/410] Channel ID: fbb60af1f1f533798757ddbfcc9d2398, L2StartNum: 108180850, L2EndNum: 108181144, L2BlockCnt, 295
[404/410] Channel ID: fccd398169ba5666298304ae3b426310, L2StartNum: 108132765, L2EndNum: 108133059, L2BlockCnt, 295
[405/410] Channel ID: fce92af6f5c4327ed3b57be0b2f6b1f6, L2StartNum: 108087658, L2EndNum: 108087957, L2BlockCnt, 300
[406/410] Channel ID: fd5c7f7e7106b6b2ebd29d0304bf3e61, L2StartNum: 108146985, L2EndNum: 108147292, L2BlockCnt, 308
[407/410] Channel ID: fe1f57497c16f7fa81fadf51ac98a2a4, L2StartNum: 108126515, L2EndNum: 108126807, L2BlockCnt, 293
[408/410] Channel ID: feb664c9fe77328435d94dbf45956278, L2StartNum: 108133355, L2EndNum: 108133654, L2BlockCnt, 300
[409/410] Channel ID: ff02d942b9988e8250a418028402468b, L2StartNum: 1081

In [14]:
!TX_ENCODE_TYPE=1  && $SPAN_BATCH_TESTER convert --in=$DB_PREFIX/channel_cache --out=$DB_PREFIX/span_batch_cache_v$TX_ENCODE_TYPE --l2=$L2_RPC --genesis-timestamp=$L2_GENESIS_TIMESTAMP --chain-id=$L2_CHAIN_ID --tx-type=$TX_ENCODE_TYPE | tail -10

[401/410] Channel ID: fa729080158c4409583baa64ca47d871, L2StartNum: 108145198, L2EndNum: 108145492, L2BlockCnt, 295
[402/410] Channel ID: fb294776caf374b6b5ad4dbba5b68e28, L2StartNum: 108113345, L2EndNum: 108113644, L2BlockCnt, 300
[403/410] Channel ID: fbb60af1f1f533798757ddbfcc9d2398, L2StartNum: 108180850, L2EndNum: 108181144, L2BlockCnt, 295
[404/410] Channel ID: fccd398169ba5666298304ae3b426310, L2StartNum: 108132765, L2EndNum: 108133059, L2BlockCnt, 295
[405/410] Channel ID: fce92af6f5c4327ed3b57be0b2f6b1f6, L2StartNum: 108087658, L2EndNum: 108087957, L2BlockCnt, 300
[406/410] Channel ID: fd5c7f7e7106b6b2ebd29d0304bf3e61, L2StartNum: 108146985, L2EndNum: 108147292, L2BlockCnt, 308
[407/410] Channel ID: fe1f57497c16f7fa81fadf51ac98a2a4, L2StartNum: 108126515, L2EndNum: 108126807, L2BlockCnt, 293
[408/410] Channel ID: feb664c9fe77328435d94dbf45956278, L2StartNum: 108133355, L2EndNum: 108133654, L2BlockCnt, 300
[409/410] Channel ID: ff02d942b9988e8250a418028402468b, L2StartNum: 1081

In [15]:
!TX_ENCODE_TYPE=2  && $SPAN_BATCH_TESTER convert --in=$DB_PREFIX/channel_cache --out=$DB_PREFIX/span_batch_cache_v$TX_ENCODE_TYPE --l2=$L2_RPC --genesis-timestamp=$L2_GENESIS_TIMESTAMP --chain-id=$L2_CHAIN_ID --tx-type=$TX_ENCODE_TYPE | tail -10

[401/410] Channel ID: fa729080158c4409583baa64ca47d871, L2StartNum: 108145198, L2EndNum: 108145492, L2BlockCnt, 295
[402/410] Channel ID: fb294776caf374b6b5ad4dbba5b68e28, L2StartNum: 108113345, L2EndNum: 108113644, L2BlockCnt, 300
[403/410] Channel ID: fbb60af1f1f533798757ddbfcc9d2398, L2StartNum: 108180850, L2EndNum: 108181144, L2BlockCnt, 295
[404/410] Channel ID: fccd398169ba5666298304ae3b426310, L2StartNum: 108132765, L2EndNum: 108133059, L2BlockCnt, 295
[405/410] Channel ID: fce92af6f5c4327ed3b57be0b2f6b1f6, L2StartNum: 108087658, L2EndNum: 108087957, L2BlockCnt, 300
[406/410] Channel ID: fd5c7f7e7106b6b2ebd29d0304bf3e61, L2StartNum: 108146985, L2EndNum: 108147292, L2BlockCnt, 308
[407/410] Channel ID: fe1f57497c16f7fa81fadf51ac98a2a4, L2StartNum: 108126515, L2EndNum: 108126807, L2BlockCnt, 293
[408/410] Channel ID: feb664c9fe77328435d94dbf45956278, L2StartNum: 108133355, L2EndNum: 108133654, L2BlockCnt, 300
[409/410] Channel ID: ff02d942b9988e8250a418028402468b, L2StartNum: 1081

span batch tester `analyze`: Analyze channels with singluar batches by comparing with corresponding span batches.

In [16]:
!TX_ENCODE_TYPE=0 && $SPAN_BATCH_TESTER analyze --in-channel=$DB_PREFIX/channel_cache --in-span-batch=$DB_PREFIX/span_batch_cache_v$TX_ENCODE_TYPE --out=$DB_PREFIX/result_v$TX_ENCODE_TYPE --chain-id=$L2_CHAIN_ID --tx-type=$TX_ENCODE_TYPE | tail -10

[399/408] Channel ID: fa729080158c4409583baa64ca47d871, CompressedReductionPercent: 7.552134 %
[400/408] Channel ID: fb294776caf374b6b5ad4dbba5b68e28, CompressedReductionPercent: 6.029229 %
[401/408] Channel ID: fbb60af1f1f533798757ddbfcc9d2398, CompressedReductionPercent: 6.801038 %
[402/408] Channel ID: fccd398169ba5666298304ae3b426310, CompressedReductionPercent: 6.601501 %
[403/408] Channel ID: fce92af6f5c4327ed3b57be0b2f6b1f6, CompressedReductionPercent: 5.112179 %
[404/408] Channel ID: fd5c7f7e7106b6b2ebd29d0304bf3e61, CompressedReductionPercent: 7.192620 %
[405/408] Channel ID: fe1f57497c16f7fa81fadf51ac98a2a4, CompressedReductionPercent: 6.151228 %
[406/408] Channel ID: feb664c9fe77328435d94dbf45956278, CompressedReductionPercent: 6.363372 %
[407/408] Channel ID: ff02d942b9988e8250a418028402468b, CompressedReductionPercent: 5.397121 %
[408/408] Channel ID: ff7de89be5bc08ae1a886fafcaf5d9ac, CompressedReductionPercent: 6.845907 %


In [17]:
!TX_ENCODE_TYPE=1 && $SPAN_BATCH_TESTER analyze --in-channel=$DB_PREFIX/channel_cache --in-span-batch=$DB_PREFIX/span_batch_cache_v$TX_ENCODE_TYPE --out=$DB_PREFIX/result_v$TX_ENCODE_TYPE --chain-id=$L2_CHAIN_ID --tx-type=$TX_ENCODE_TYPE | tail -10

[399/408] Channel ID: fa729080158c4409583baa64ca47d871, CompressedReductionPercent: 4.504091 %
[400/408] Channel ID: fb294776caf374b6b5ad4dbba5b68e28, CompressedReductionPercent: 2.985339 %
[401/408] Channel ID: fbb60af1f1f533798757ddbfcc9d2398, CompressedReductionPercent: 3.902365 %
[402/408] Channel ID: fccd398169ba5666298304ae3b426310, CompressedReductionPercent: 3.586205 %
[403/408] Channel ID: fce92af6f5c4327ed3b57be0b2f6b1f6, CompressedReductionPercent: 2.596872 %
[404/408] Channel ID: fd5c7f7e7106b6b2ebd29d0304bf3e61, CompressedReductionPercent: 4.031155 %
[405/408] Channel ID: fe1f57497c16f7fa81fadf51ac98a2a4, CompressedReductionPercent: 2.840200 %
[406/408] Channel ID: feb664c9fe77328435d94dbf45956278, CompressedReductionPercent: 3.488432 %
[407/408] Channel ID: ff02d942b9988e8250a418028402468b, CompressedReductionPercent: 2.442320 %
[408/408] Channel ID: ff7de89be5bc08ae1a886fafcaf5d9ac, CompressedReductionPercent: 3.560317 %


In [18]:
!TX_ENCODE_TYPE=2 && $SPAN_BATCH_TESTER analyze --in-channel=$DB_PREFIX/channel_cache --in-span-batch=$DB_PREFIX/span_batch_cache_v$TX_ENCODE_TYPE --out=$DB_PREFIX/result_v$TX_ENCODE_TYPE --chain-id=$L2_CHAIN_ID --tx-type=$TX_ENCODE_TYPE | tail -10

[399/408] Channel ID: fa729080158c4409583baa64ca47d871, CompressedReductionPercent: 9.989832 %
[400/408] Channel ID: fb294776caf374b6b5ad4dbba5b68e28, CompressedReductionPercent: 8.929760 %
[401/408] Channel ID: fbb60af1f1f533798757ddbfcc9d2398, CompressedReductionPercent: 9.655904 %
[402/408] Channel ID: fccd398169ba5666298304ae3b426310, CompressedReductionPercent: 9.384179 %
[403/408] Channel ID: fce92af6f5c4327ed3b57be0b2f6b1f6, CompressedReductionPercent: 7.596212 %
[404/408] Channel ID: fd5c7f7e7106b6b2ebd29d0304bf3e61, CompressedReductionPercent: 9.869628 %
[405/408] Channel ID: fe1f57497c16f7fa81fadf51ac98a2a4, CompressedReductionPercent: 9.153978 %
[406/408] Channel ID: feb664c9fe77328435d94dbf45956278, CompressedReductionPercent: 9.191252 %
[407/408] Channel ID: ff02d942b9988e8250a418028402468b, CompressedReductionPercent: 8.368134 %
[408/408] Channel ID: ff7de89be5bc08ae1a886fafcaf5d9ac, CompressedReductionPercent: 9.723770 %


span batch tester `fetch`: Fetch consecutive singular batches.

In [19]:
!$SPAN_BATCH_TESTER fetch --l2 $L2_RPC --start=$ACC_L2_START --end=$ACC_L2_END --concurrent-requests=100 --out=$DB_PREFIX/singular_batches_cache | tail -10

Fetched L2 block:  107880992
Fetched L2 block:  107880996
Fetched L2 block:  107880997
Fetched L2 block:  107880995
Fetched L2 block:  107880993
Fetched L2 block:  107880994
Fetched L2 block:  107880999
Fetched L2 block:  107880998
Fetched v0 batches in range [107880000,107881000).
Fetch Config: Chain ID: 10.


span batch tester `merge`: Merge singular batches:

This command will analyze `start - end` number of batches, Comparing uncompressed size of below two batches:
- For `i` in range  `0` to `start - end - 1`
  - singular batch from number `start` to `start + i`, inclusive.
  - span batch from number `start` to `start + i`, inclusive.

Upper command will make a result json file which has name of the form `{start_number}_{end_number}.json`.

In [12]:
!$SPAN_BATCH_TESTER merge --start=$ACC_L2_START --end=$ACC_L2_END --l2 $L2_RPC --genesis-timestamp=$L2_GENESIS_TIMESTAMP --in=$DB_PREFIX/singular_batches_cache --out=$DB_PREFIX/merge_result | tail -10

Merged v0 batches in range [107880000,107881000).


span batch tester `format`: Compare different tx encodings for span batch.


In [21]:
!$SPAN_BATCH_TESTER format --permutation 0123465 --in-span-batch=$DB_PREFIX/span_batch_cache_v2 --out=$DB_PREFIX/format_result --chain-id=$L2_CHAIN_ID | tail -10

Reduction Percentage: 0.007508 %
[405/408] cnt: 324, delta: 94, deltasum: 37321, originalCompressedSizeSum: 212245285
Reduction Percentage: 0.018305 %
[406/408] cnt: 325, delta: 41, deltasum: 37362, originalCompressedSizeSum: 212670092
Reduction Percentage: 0.009651 %
[407/408] cnt: 325, delta: -80, deltasum: 37282, originalCompressedSizeSum: 213280502
Reduction Percentage: -0.013106 %
[408/408] cnt: 326, delta: 157, deltasum: 37439, originalCompressedSizeSum: 213694685
Reduction Percentage: 0.037906 %
Final Reduction Percentage: 0.017520 %


In [22]:
!$SPAN_BATCH_TESTER format --permutation 0134625 --in-span-batch=$DB_PREFIX/span_batch_cache_v2 --out=$DB_PREFIX/format_result --chain-id=$L2_CHAIN_ID | tail -10

Reduction Percentage: -0.046603 %
[405/408] cnt: 262, delta: 183, deltasum: 45856, originalCompressedSizeSum: 212245285
Reduction Percentage: 0.035636 %
[406/408] cnt: 262, delta: -170, deltasum: 45686, originalCompressedSizeSum: 212670092
Reduction Percentage: -0.040018 %
[407/408] cnt: 262, delta: -325, deltasum: 45361, originalCompressedSizeSum: 213280502
Reduction Percentage: -0.053243 %
[408/408] cnt: 263, delta: 5, deltasum: 45366, originalCompressedSizeSum: 213694685
Reduction Percentage: 0.001207 %
Final Reduction Percentage: 0.021229 %


In [23]:
!$SPAN_BATCH_TESTER format --permutation 3041265 --in-span-batch=$DB_PREFIX/span_batch_cache_v2 --out=$DB_PREFIX/format_result --chain-id=$L2_CHAIN_ID | tail -10

Reduction Percentage: 0.014499 %
[405/408] cnt: 350, delta: 307, deltasum: 97958, originalCompressedSizeSum: 212245285
Reduction Percentage: 0.059782 %
[406/408] cnt: 351, delta: 13, deltasum: 97971, originalCompressedSizeSum: 212670092
Reduction Percentage: 0.003060 %
[407/408] cnt: 351, delta: -35, deltasum: 97936, originalCompressedSizeSum: 213280502
Reduction Percentage: -0.005734 %
[408/408] cnt: 352, delta: 128, deltasum: 98064, originalCompressedSizeSum: 213694685
Reduction Percentage: 0.030904 %
Final Reduction Percentage: 0.045890 %


In [24]:
!$SPAN_BATCH_TESTER format --permutation 0125634 --in-span-batch=$DB_PREFIX/span_batch_cache_v2 --out=$DB_PREFIX/format_result --chain-id=$L2_CHAIN_ID | tail -10

Reduction Percentage: 0.052040 %
[405/408] cnt: 365, delta: 478, deltasum: 96751, originalCompressedSizeSum: 212245285
Reduction Percentage: 0.093081 %
[406/408] cnt: 366, delta: 287, deltasum: 97038, originalCompressedSizeSum: 212670092
Reduction Percentage: 0.067560 %
[407/408] cnt: 366, delta: -14, deltasum: 97024, originalCompressedSizeSum: 213280502
Reduction Percentage: -0.002294 %
[408/408] cnt: 367, delta: 141, deltasum: 97165, originalCompressedSizeSum: 213694685
Reduction Percentage: 0.034043 %
Final Reduction Percentage: 0.045469 %


In [25]:
!$SPAN_BATCH_TESTER format --permutation 0324165 --in-span-batch=$DB_PREFIX/span_batch_cache_v2 --out=$DB_PREFIX/format_result --chain-id=$L2_CHAIN_ID | tail -10

Reduction Percentage: -0.048415 %
[405/408] cnt: 171, delta: 56, deltasum: -13307, originalCompressedSizeSum: 212245285
Reduction Percentage: 0.010905 %
[406/408] cnt: 171, delta: -169, deltasum: -13476, originalCompressedSizeSum: 212670092
Reduction Percentage: -0.039783 %
[407/408] cnt: 171, delta: -405, deltasum: -13881, originalCompressedSizeSum: 213280502
Reduction Percentage: -0.066349 %
[408/408] cnt: 171, delta: -83, deltasum: -13964, originalCompressedSizeSum: 213694685
Reduction Percentage: -0.020039 %
Final Reduction Percentage: -0.006535 %


In [26]:
!$SPAN_BATCH_TESTER format --permutation 3041256 --in-span-batch=$DB_PREFIX/span_batch_cache_v2 --out=$DB_PREFIX/format_result --chain-id=$L2_CHAIN_ID | tail -10

Reduction Percentage: 0.013722 %
[405/408] cnt: 402, delta: 305, deltasum: 87474, originalCompressedSizeSum: 212245285
Reduction Percentage: 0.059393 %
[406/408] cnt: 403, delta: 32, deltasum: 87506, originalCompressedSizeSum: 212670092
Reduction Percentage: 0.007533 %
[407/408] cnt: 404, delta: 52, deltasum: 87558, originalCompressedSizeSum: 213280502
Reduction Percentage: 0.008519 %
[408/408] cnt: 405, delta: 30, deltasum: 87588, originalCompressedSizeSum: 213694685
Reduction Percentage: 0.007243 %
Final Reduction Percentage: 0.040987 %


In [27]:
!$SPAN_BATCH_TESTER format --permutation 3401256 --in-span-batch=$DB_PREFIX/span_batch_cache_v2 --out=$DB_PREFIX/format_result --chain-id=$L2_CHAIN_ID | tail -10

Reduction Percentage: 0.013463 %
[405/408] cnt: 402, delta: 305, deltasum: 87533, originalCompressedSizeSum: 212245285
Reduction Percentage: 0.059393 %
[406/408] cnt: 403, delta: 33, deltasum: 87566, originalCompressedSizeSum: 212670092
Reduction Percentage: 0.007768 %
[407/408] cnt: 404, delta: 52, deltasum: 87618, originalCompressedSizeSum: 213280502
Reduction Percentage: 0.008519 %
[408/408] cnt: 405, delta: 30, deltasum: 87648, originalCompressedSizeSum: 213694685
Reduction Percentage: 0.007243 %
Final Reduction Percentage: 0.041016 %


In [28]:
!$SPAN_BATCH_TESTER format --permutation 6051234 --in-span-batch=$DB_PREFIX/span_batch_cache_v2 --out=$DB_PREFIX/format_result --chain-id=$L2_CHAIN_ID | tail -10

Reduction Percentage: 0.028997 %
[405/408] cnt: 352, delta: 380, deltasum: 111833, originalCompressedSizeSum: 212245285
Reduction Percentage: 0.073998 %
[406/408] cnt: 353, delta: 361, deltasum: 112194, originalCompressedSizeSum: 212670092
Reduction Percentage: 0.084980 %
[407/408] cnt: 354, delta: 115, deltasum: 112309, originalCompressedSizeSum: 213280502
Reduction Percentage: 0.018840 %
[408/408] cnt: 355, delta: 79, deltasum: 112388, originalCompressedSizeSum: 213694685
Reduction Percentage: 0.019074 %
Final Reduction Percentage: 0.052593 %


In [29]:
!$SPAN_BATCH_TESTER format --permutation 0132456 --in-span-batch=$DB_PREFIX/span_batch_cache_v2 --out=$DB_PREFIX/format_result --chain-id=$L2_CHAIN_ID | tail -10

Reduction Percentage: -0.051004 %
[405/408] cnt: 109, delta: -4, deltasum: -44113, originalCompressedSizeSum: 212245285
Reduction Percentage: -0.000779 %
[406/408] cnt: 109, delta: -196, deltasum: -44309, originalCompressedSizeSum: 212670092
Reduction Percentage: -0.046139 %
[407/408] cnt: 109, delta: -326, deltasum: -44635, originalCompressedSizeSum: 213280502
Reduction Percentage: -0.053407 %
[408/408] cnt: 109, delta: -223, deltasum: -44858, originalCompressedSizeSum: 213694685
Reduction Percentage: -0.053841 %
Final Reduction Percentage: -0.020992 %


In [30]:
!$SPAN_BATCH_TESTER format --permutation 0352614 --in-span-batch=$DB_PREFIX/span_batch_cache_v2 --out=$DB_PREFIX/format_result --chain-id=$L2_CHAIN_ID | tail -10

Reduction Percentage: -0.017606 %
[405/408] cnt: 116, delta: 110, deltasum: -51919, originalCompressedSizeSum: 212245285
Reduction Percentage: 0.021420 %
[406/408] cnt: 116, delta: -269, deltasum: -52188, originalCompressedSizeSum: 212670092
Reduction Percentage: -0.063323 %
[407/408] cnt: 116, delta: -674, deltasum: -52862, originalCompressedSizeSum: 213280502
Reduction Percentage: -0.110418 %
[408/408] cnt: 116, delta: -321, deltasum: -53183, originalCompressedSizeSum: 213694685
Reduction Percentage: -0.077502 %
Final Reduction Percentage: -0.024887 %
